# Extract from TMDB with API

In [25]:
import pandas as pd
import requests
import math
import re
import os
from dotenv import load_dotenv
from pathlib import Path

In [26]:
load_dotenv('../.env')

# Récupérer le token
TMDB_API_TOKEN = os.getenv('TMDB_API_TOKEN')

In [27]:
BASE_DIR = Path().resolve().parent
DATA_DIR = BASE_DIR / 'data'

print(f"Répertoire de sauvegarde: {DATA_DIR}\n")

Répertoire de sauvegarde: C:\Users\Melvin\Desktop\DATA\PORTFOLIO\Recommandation de films\data



Extraction des films

In [28]:
# defineing a function to check if the title is in latin characters

def is_latin_title(title):
    """Vérifie si le titre est en caractères latins"""
    if not title:
        return False
    
    non_latin = r'[\u4e00-\u9fff\u3040-\u309f\u30a0-\u30ff\uac00-\ud7af\u0400-\u04ff\u0600-\u06ff]'
    return not re.search(non_latin, title)

In [ ]:
url_base = "https://api.themoviedb.org/3/discover/movie?include_adult=true&include_video=false&language=fr-FR&page=1&sort_by=popularity.desc"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {TMDB_API_TOKEN}"
}

params = {
    'language': 'fr-FR'
}

movies = []

page = 1

max_pages = 500

while page <= max_pages:
    params['page'] = page
    response = requests.get(url_base, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        for movie in data['results']:
            title = movie.get('title', '')
            
            if is_latin_title(title):
                movies.append([
                    movie.get('id', math.nan),
                    title,
                    movie.get('release_date', math.nan),
                    movie.get('genre_ids', math.nan),
                    movie.get('vote_average', math.nan),
                    movie.get('popularity', math.nan),
                    movie.get('overview', math.nan),
                    movie.get('poster_path', math.nan)
                ])
        
        page += 1
    else:
        print(f"Erreur lors de la récupération de la page {page}: Status code {response.status_code}")
        break

print(f"Total de films récupérés: {len(movies)}")

Erreur lors de la récupération de la page 501: Status code 400
Total de films récupérés: 9727


In [30]:
df = pd.DataFrame(movies)

Extraction des genres

In [31]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=fr"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    genres_data = response.json()
    genres_list = genres_data['genres']
    
    df_genres = pd.DataFrame(genres_list)
    
    print(df_genres)
else:
    print(f"Erreur dans la requête API: {response.status_code}")

       id             name
0      28           Action
1      12         Aventure
2      16        Animation
3      35          Comédie
4      80            Crime
5      99     Documentaire
6      18            Drame
7   10751         Familial
8      14      Fantastique
9      36         Histoire
10     27          Horreur
11  10402          Musique
12   9648          Mystère
13  10749          Romance
14    878  Science-Fiction
15  10770         Téléfilm
16     53         Thriller
17  10752           Guerre
18     37          Western


Remplacement des genres int par str 

In [32]:
genre_dict = pd.Series(df_genres.name.values, index=df_genres.id).to_dict()

In [33]:
def replace_ids_with_names(id_list, genre_dict):
    return [genre_dict.get(i) for i in id_list]

df['genre'] = df[3].apply(lambda x: replace_ids_with_names(x, genre_dict))

In [34]:
df.drop(columns=3, inplace=True)

In [35]:
columns = 'ID', 'Titre', 'Date de sortie', 'Note', 'Popularité', 'Synopsis', 'Affiche', 'Genre'

df.columns = columns

In [36]:
df = df[['ID', 'Titre', 'Date de sortie', 'Genre', 'Note', 'Popularité', 'Synopsis', 'Affiche']]

In [37]:
df['Genre'] = df['Genre'].apply(lambda x: ', '.join(map(str, x)))

In [38]:
df.head(5)

,ID,Titre,Date de sortie,Genre,Note,Popularité,Synopsis,Affiche
0,840464,Greenland : Migration,2026-01-07,"Aventure, Thriller, Science-Fiction",6.567,499.8774,Après l'impact dévastateur d'une comète qui a ...,/9eaHOemq2Ch5rYPJ0uXcipZUYpS.jpg
1,1368166,La Femme de ménage,2025-12-18,"Mystère, Thriller",7.178,473.7526,"En quête d’un nouveau départ, Millie accepte u...",/28Dn93nuxH8PmDLUVQLCjLNVxKD.jpg
2,1168190,Team Démolition,2026-01-28,"Action, Comédie, Crime, Mystère",6.802,418.4280,"Deux demi-frères brouillés, Jonny et James, se...",/ec0bwYUEYBhcHutx4nIShLzX7Dl.jpg
3,1084242,Zootopie 2,2025-11-26,"Animation, Comédie, Aventure, Familial, Mystère",7.600,290.4141,Judy Hopps et Nick Wilde explorent à nouveau Z...,/hBI7Wrps6tDjhEzBxJgoPLhbmT1.jpg
4,1419406,The Shadow's Edge,2025-08-16,"Action, Crime, Drame, Thriller",7.202,262.5255,Un mystérieux mafieux et ses 7 fils adoptifs m...,/t1PFVsGYdUHtPv0Xowoc9b4PAap.jpg


In [41]:
movies_path = DATA_DIR / 'df_movies.csv'
df.to_csv(movies_path, index=False, encoding='utf-8')
print(f"Films sauvegardés à l'emplacement : {movies_path}")
print(f"Nombre de films sauvegardés : {len(df):,}")

Films sauvegardés à l'emplacement : C:\Users\Melvin\Desktop\DATA\PORTFOLIO\Recommandation de films\data\df_movies.csv
Nombre de films sauvegardés : 9,727
